In [1]:
# https://www.bilibili.com/video/BV18T411t7h6/?spm_id_from=333.788&vd_source=d278a161bb4aef15e279eee0993f0b2e
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModel

abs_path = os.getcwd() + "/../../../"
abs_path = abs_path.replace("/", "\\") if os.name == "nt" else path
model_path = abs_path + "coder\\model\\rbt3"
print(abs_path)


d:\Anaconda\envs\LLms\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Anaconda\envs\LLms\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


g:\AI\AI-project\LLM-experiment\coder\SFT\rbt3\..\..\..\


In [2]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        path = abs_path + "dataset/rbt3/ChnSentiCorp_htl_all.csv"
        self.data = pd.read_csv(path)
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)
        

In [3]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [4]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, [0.9, 0.1])
print(len(train_set), len(valid_set))

6989 776


In [16]:
import torch 
tokenizer = AutoTokenizer.from_pretrained(abs_path + "coder\\model\\rbt3", local_files_only = True)

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs
    

In [17]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True, collate_fn=collate_func)
valid_loader = DataLoader(dataset=valid_set, batch_size=64, shuffle=True, collate_fn=collate_func)

In [18]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained(model_path)
optimizer = Adam(model.parameters(), lr=2e-5)
if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at g:\AI\AI-project\LLM-experiment\coder\SFT\rbt3\..\..\..\coder\model\rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Step7 训练与验证

In [19]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch['labels'].long()).float().sum()
    return acc_num / len(valid_set)

def train(epoch=5, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f'Epoch: {ep}, Step: {global_step}, Loss: {output.loss.item()}')
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

In [20]:
train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 8.00 GiB total capacity; 147.50 MiB already allocated; 6.71 GiB free; 164.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF